In [ ]:
# TODO add license

In [ ]:
from __future__ import print_function

import larq as lq
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle

from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.utils import np_utils
import tensorflow as tf

np.random.seed(1337)

### Dataset

In [ ]:
train_df = pd.read_csv('datasets/UNSW_NB15/UNSW_NB15_training-set.csv', delimiter=',')
test_df = pd.read_csv('datasets/UNSW_NB15/UNSW_NB15_testing-set.csv', delimiter=',')

In [ ]:
data_df = train_df.append(test_df)

In [ ]:
data_df = data_df.drop(['id'], axis=1)
cols = ['proto', 'service', 'state']
for col in cols:
    data_df[col] =  data_df[col].astype('category')
    data_df[col] =  data_df[col].cat.codes

### Feature selection

In [ ]:
selected_columns = [
    'dur',
    'proto',
    'sbytes', 'dbytes',
    'sttl', 'dttl',
    'sload', 'dload',
    'spkts', 'dpkts',
    'smean', 'dmean',
    'sinpkt', 'dinpkt',
    'tcprtt', 'synack', 'ackdat',
    'ct_src_ltm', 'ct_dst_ltm',
    'ct_dst_src_ltm',

    'attack_cat'
]

In [ ]:
data_df = data_df[selected_columns]
X = data_df[data_df.columns[:20]].values
Y = data_df[data_df.columns[20]].values

### Data binarization

In [ ]:
Xint = X.astype("int")

In [ ]:
size_in_bits = {
    'dur': 8,
    'proto': 8,
    'sbytes': 16, 'dbytes': 16,
    'sttl': 8, 'dttl': 8,
    'sload': 24, 'dload': 24,
    'spkts': 16, 'dpkts': 16,
    'smean': 16, 'dmean': 16,
    'sinpkt': 16, 'dinpkt': 16,
    'tcprtt': 8, 'synack': 8, 'ackdat': 8,
    'ct_src_ltm': 8, 'ct_dst_ltm': 8,
    'ct_dst_src_ltm': 8,
}

sum(size_in_bits.values())

In [ ]:
Xbin = np.zeros( (Xint.shape[0], sum(size_in_bits.values())) )
for i, feature_row in enumerate(Xint):
    # the index at which the next binary value should be written
    write_ptr = 0
    for j, column_val in enumerate(feature_row):
        # Transforming in KB sbytes, dbytes, sload, dload
        if j in [2,3,6,7]:
            column_val = int(column_val/1000) 
        # Setting to maximum any value above the max given the number of b
        if (column_val > 2**size_in_bits[selected_columns[j]] - 1):
            column_val = 2**size_in_bits[selected_columns[j]] - 1
        tmp = list(bin(column_val)[2:])
        tmp = [int(x) for x in tmp]
        # zero padding to the left
        tmp = [0]*(size_in_bits[selected_columns[j]] - len(tmp)) + tmp
        for k, bin_val in enumerate(tmp):
            Xbin[i,write_ptr] = bin_val
            write_ptr += 1

In [ ]:
# DT/RF dataset
Y__2_classes = Y.copy()
Y__2_classes[Y == 'Normal'] = 0
Y__2_classes[Y != 'Normal'] = 1
Y__2_classes = Y__2_classes.astype('int')

# BNN dataset
Xbin[Xbin == 0] = -1
X_bin = Xbin

Y_cat__2_classes = np_utils.to_categorical(Y__2_classes)

In [ ]:
def plot_confusion_matrix_wo_clf(y_true, y_pred, class_names, normalize='true'):
    cm = confusion_matrix(y_true, y_pred, normalize=normalize)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    fig, ax = plt.subplots(figsize=(6, 6))
    if normalize == 'true':
        disp.plot(cmap=plt.cm.Blues, ax=ax, values_format='.2f')
    else:
        disp.plot(cmap=plt.cm.Blues, ax=ax)
    
    plt.show()

In [ ]:
def metrics_binary_dataset(clf, X_test, y_test, y_pred, y_score, is_bnn=False):
    
    if is_bnn:
        # Make y_test 1D
        y_test = np.argmax(y_test, axis=1)
        
    plot_confusion_matrix_wo_clf(y_test, y_pred, ['0','1'])
   
    a = accuracy_score(y_test, y_pred)
    p = precision_score(y_test, y_pred)
    r = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='macro')

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    tpr = r
    fpr = fp / (fp+tn)
    fnr = fn / (fn+tp)
    tnr = tn / (tn+fp)
    
    y_score = to_categorical(y_pred)
    fpr_, tpr_, _ = roc_curve(y_test, y_score[:, 1])
    roc_auc = auc(fpr_, tpr_)

    return a, p, r, tpr, fpr, fnr, f1, roc_auc

In [ ]:
def build_bnn_model(neurons, 
                    input_shape, 
                    last_act="softmax", 
                    learning_rate=0.0001, 
                    loss='squared_hinge'):
    
    kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip")

    model = tf.keras.models.Sequential()
    model.add(lq.layers.QuantDense(neurons[0], use_bias=False,
                                   input_quantizer="ste_sign",
                                   kernel_quantizer="ste_sign",
                                   kernel_constraint="weight_clip",
                                   input_shape=(input_shape,) ) )
    model.add(tf.keras.layers.BatchNormalization(scale=False, momentum=0.9))
    model.add(lq.layers.QuantDense(neurons[1], use_bias=False, **kwargs))
    model.add(tf.keras.layers.BatchNormalization(scale=False, momentum=0.9))
    model.add(lq.layers.QuantDense(neurons[2], use_bias=False, activation=last_act, **kwargs))

    # lq.models.summary(model)
    
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])
    
    return model

In [ ]:
depths_range = [3, 6, 9]
estimators_range = [5]

bnn_models = [
    [32, 16, 2],
    [64, 32, 2],
    [128, 64, 2]
]

In [ ]:
batch_size = 256
num_folds = 5
train_epochs = 15

In [ ]:
accuracy_store_sec = {}
precision_store_sec = {}
recall_store_sec = {}
tpr_store_sec = {}
fpr_store_sec = {}
fnr_store_sec = {}
f1_store_sec = {}
roc_auc_store_sec = {}

########################################

skf = StratifiedShuffleSplit(n_splits=num_folds, random_state=0)

for depth in depths_range:    
    label = 'dt__depth_%d' % (depth)
    accuracy_store_sec[label] = np.zeros(num_folds)
    precision_store_sec[label] = np.zeros(num_folds)
    recall_store_sec[label] = np.zeros(num_folds)
    tpr_store_sec[label] = np.zeros(num_folds)
    fpr_store_sec[label] = np.zeros(num_folds)
    fnr_store_sec[label] = np.zeros(num_folds)
    f1_store_sec[label] = np.zeros(num_folds)
    roc_auc_store_sec[label] = np.zeros(num_folds)
    
    fold_idx = 0
    for train_index, test_index in skf.split(X, Y):
        print('DT: depth=%d, fold_idx=%d' % (depth, fold_idx))
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y__2_classes[train_index], Y__2_classes[test_index]
                        
        dt = DecisionTreeClassifier(criterion='entropy', max_depth=depth, random_state=0)
        dt = dt.fit(X_train, y_train)
        y_pred = dt.predict(X_test)
        y_score = dt.predict_proba(X_test)

        a, p, r, tpr, fpr, fnr, f1, roc_auc = metrics_binary_dataset(dt, X_test, y_test, y_pred, y_score)
        accuracy_store_sec[label][fold_idx] = a
        precision_store_sec[label][fold_idx] = p
        recall_store_sec[label][fold_idx] = r
        tpr_store_sec[label][fold_idx] = tpr
        fpr_store_sec[label][fold_idx] = fpr
        fnr_store_sec[label][fold_idx] = fnr
        f1_store_sec[label][fold_idx] = f1
        roc_auc_store_sec[label][fold_idx] = roc_auc

        fold_idx += 1

        print('-'*40)
    
    print('='*80)

########################################################################################

for depth in depths_range:
    for estimators in estimators_range:
        label = 'rf__depth_%d__estimators_%d' % (depth, estimators)
        accuracy_store_sec[label] = np.zeros(num_folds)
        precision_store_sec[label] = np.zeros(num_folds)
        recall_store_sec[label] = np.zeros(num_folds)
        tpr_store_sec[label] = np.zeros(num_folds)
        fpr_store_sec[label] = np.zeros(num_folds)
        fnr_store_sec[label] = np.zeros(num_folds)
        f1_store_sec[label] = np.zeros(num_folds)
        roc_auc_store_sec[label] = np.zeros(num_folds)

        fold_idx = 0
        for train_index, test_index in skf.split(X, Y):
            print('RF: depth=%d, estimators=%d, fold_idx=%d' % (depth, estimators, fold_idx))
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y__2_classes[train_index], Y__2_classes[test_index]

            rf = RandomForestClassifier(criterion='entropy', max_depth=depth, n_estimators=estimators, random_state=0)
            rf = rf.fit(X_train, y_train)
            y_pred = rf.predict(X_test)
            y_score = rf.predict_proba(X_test)

            a, p, r, tpr, fpr, fnr, f1, roc_auc = metrics_binary_dataset(rf, X_test, y_test, y_pred, y_score)
            accuracy_store_sec[label][fold_idx] = a
            precision_store_sec[label][fold_idx] = p
            recall_store_sec[label][fold_idx] = r
            tpr_store_sec[label][fold_idx] = tpr
            fpr_store_sec[label][fold_idx] = fpr
            fnr_store_sec[label][fold_idx] = fnr
            f1_store_sec[label][fold_idx] = f1
            roc_auc_store_sec[label][fold_idx] = roc_auc
        
            fold_idx += 1

            print('-'*40)
            
        print('='*40)

    print('='*80)

########################################################################################

for neurons in bnn_models:
    label = 'bnn__%s' % ('_'.join(map(str, neurons)))
    accuracy_store_sec[label] = np.zeros(num_folds)
    precision_store_sec[label] = np.zeros(num_folds)
    recall_store_sec[label] = np.zeros(num_folds)
    tpr_store_sec[label] = np.zeros(num_folds)
    fpr_store_sec[label] = np.zeros(num_folds)
    fnr_store_sec[label] = np.zeros(num_folds)
    f1_store_sec[label] = np.zeros(num_folds)
    roc_auc_store_sec[label] = np.zeros(num_folds)
    
    fold_idx = 0
    for train_index, test_index in skf.split(X, Y):
        print('BNN', neurons ,', fold_idx=%d' % (fold_idx))
        X_train, X_test = X_bin[train_index], X_bin[test_index]
        y_train, y_test = Y_cat__2_classes[train_index], Y_cat__2_classes[test_index]
                        
        model = build_bnn_model(neurons, X_bin.shape[1])   
        fname = 'bnn__sec_StratifiedShuffleSplit__%s__fold%d.h5' % ('_'.join(map(str, neurons)), fold_idx)
        
        model_checkpoint_callback = ModelCheckpoint(
            filepath='models/' + fname,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1)
        
        if not os.path.isfile('models/' + fname):
            train_history = model.fit(X_train, y_train, 
                              batch_size=batch_size, 
                              epochs=train_epochs,
                              verbose=1,
                              validation_data=(X_test, y_test),
                              callbacks=[model_checkpoint_callback])
                    
            # Reload best weights
            model.load_weights('models/' + fname)
        else:
            # Reload stored weights
            print('Loading models/' + fname)
            model.load_weights('models/' + fname)

        y_pred = model.predict_classes(X_test)
        y_score = model.predict_proba(X_test)
        
        a, p, r, tpr, fpr, fnr, f1, roc_auc = metrics_binary_dataset(model, X_test, y_test, y_pred, y_score, is_bnn=True)
        accuracy_store_sec[label][fold_idx] = a
        precision_store_sec[label][fold_idx] = p
        recall_store_sec[label][fold_idx] = r
        tpr_store_sec[label][fold_idx] = tpr
        fpr_store_sec[label][fold_idx] = fpr
        fnr_store_sec[label][fold_idx] = fnr
        f1_store_sec[label][fold_idx] = f1
        roc_auc_store_sec[label][fold_idx] = roc_auc
        
        fold_idx += 1
        
        print('-'*40)
    
    print('='*80)

In [ ]:
for store,metric in zip([accuracy_store_sec, precision_store_sec, recall_store_sec,
                         fnr_store_sec, fpr_store_sec, f1_store_sec, roc_auc_store_sec],
                        ['Accuracy', 'Precision', 'Recall', 'FNR', 'FPR', 'F1-score', 'ROC-AUC']):
    print('[%s]' % metric)
    for key in store:
        print('%s: %.1f ± %.1f' % (key, 100*store[key].mean(), 100*store[key].std()))
    print()